In [1]:
cd /home/pi/Downloads/PiCamera_ObjectDetection/model

/home/pi/Downloads/PiCamera_ObjectDetection/model


In [2]:
!protoc object_detection/protos/*.proto --python_out=.

In [3]:
import io
import time
import numpy as np
import psutil
import picamera
import tensorflow as tf

from collections import defaultdict
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import warnings
warnings.filterwarnings('ignore')

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [4]:
detection_model = tf.saved_model.load(str("/home/pi/Downloads/PiCamera_ObjectDetection/model/ssd_mobilenet_v1_coco_2017_11_17/saved_model"))
detection_model = detection_model.signatures['serving_default']
PATH_TO_LABELS = '/home/pi/Downloads/PiCamera_ObjectDetection/model/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [5]:
def run_inference_for_single_image(model, image):
    image = np.asarray(image)
    input_tensor = tf.convert_to_tensor(image)
    input_tensor = input_tensor[tf.newaxis,...]
    output_dict = model(input_tensor)
    num_detections = int(output_dict.pop('num_detections'))
    output_dict = {key:value[0, :num_detections].numpy() 
        for key,value in output_dict.items()}
    output_dict['num_detections'] = num_detections
    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
    if 'detection_masks' in output_dict:
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            output_dict['detection_masks'], output_dict['detection_boxes'],
            image.shape[0], image.shape[1])      
        detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,tf.uint8)
        output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
    return output_dict

In [6]:
def dist_cal(boxes,resolution):
    (width,height)=resolution

    y1=boxes[0][0]*height
    x1=boxes[0][1]*width
    y2=boxes[0][2]*height
    x2=boxes[0][3]*width
    y3=boxes[1][0]*height
    x3=boxes[1][1]*width
    y4=boxes[1][2]*height
    x4=boxes[1][3]*width

    x=(x1+x2)/2 - (x3+x4)/2
    y=(y1+y2)/2 - (y3+y4)/2
    dist = np.sqrt( (x)**2 + (y)**2 )
    
    return ([(x1+x2)/2,(y1+y2)/2,(x3+x4)/2,(y3+y4)/2],dist)

In [7]:
def show_inference(model, image_path,class_id):
    image_np = np.array(Image.open(image_path))
    output_dict = run_inference_for_single_image(model, image_np)
    boxes = []
    classes = []
    scores = []
    for i,x in enumerate(output_dict['detection_classes']):
        if x==class_id and output_dict['detection_scores'][i] > 0.5:
            classes.append(x)
            boxes.append(output_dict['detection_boxes'][i])
            scores.append(output_dict['detection_scores'][i])
    boxes = np.array(boxes)
    classes = np.array(classes)
    scores = np.array(scores)
    vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      boxes,
      classes,
      scores,
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=2)
  
    #display(Image.fromarray(image_np))
    myimage=Image.fromarray(image_np)
    return boxes

In [8]:
camera=picamera.PiCamera()

In [28]:
camera.close()

In [9]:
import numpy as np
import math
camera.resolution=(320,240)
camera.start_preview(fullscreen=False,window=(100,100,480,320))
camera.annotate_text="Covid 19 Fighter"

(VIDEO_WIDTH,VIDEO_HEIGHT) = camera.resolution
mybox = Image.new("RGB", (VIDEO_WIDTH, VIDEO_HEIGHT))
overlay = camera.add_overlay(mybox.tobytes(),fullscreen=False,window=(100,100,480,320),layer = 3, alpha = 100) 

    
i=0
try:
    while i<100:
        stream = io.BytesIO()
        camera.capture(stream,format='jpeg')
        stream.seek(0)
        boxes=show_inference(detection_model, stream, 44)
        img = mybox.copy()
        myboxPixels = img.load()
        
        if len(boxes)==2:
            for box in boxes:
                y1=math.floor(box[0]*VIDEO_HEIGHT)
                x1=math.floor(box[1]*VIDEO_WIDTH)
                y2=math.floor(box[2]*VIDEO_HEIGHT)-1
                x2=math.floor(box[3]*VIDEO_WIDTH)-1
                print(x1,y1,x2,y2)
                for x in range (x1, x2):
                    myboxPixels[x, y1] = (255, 255, 0)
                    myboxPixels[x, y2] = (255, 255, 0)
                for y in range(y1, y2):
                    myboxPixels[x1, y] = (255, 255, 0)
                    myboxPixels[x2, y] = (255, 255, 0)
            (coord,pixel_distance)=dist_cal(boxes,camera.resolution)
            print(pixel_distance)
            
            draw=ImageDraw.Draw(img)
            draw.line([(coord[0],coord[1]),(coord[2],coord[3])],fill=(255, 255, 0),width=0)
            draw.text((coord[0],coord[1]),str(round(pixel_distance,2)),(255, 255, 0))
                
        overlay.update(img.tobytes())
        print(i)
        i=i+1

        time.sleep(0.2)
    camera.remove_overlay(overlay)
    camera.stop_preview()
except KeyboardInterrupt:
    camera.remove_overlay(overlay)
    camera.stop_preview()
    
    print("Camera Stopped")


2021-10-31 17:33:19.156574: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 230400 exceeds 10% of system memory.
2021-10-31 17:33:22.350642: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 262144 exceeds 10% of system memory.
2021-10-31 17:33:22.351825: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 524288 exceeds 10% of system memory.
2021-10-31 17:33:22.353069: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 1048576 exceeds 10% of system memory.
2021-10-31 17:33:22.354639: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 1048576 exceeds 10% of system memory.


0
1


PiCameraMMALError: no buffers available: Resource temporarily unavailable; try again later

2


PiCameraMMALError: no buffers available: Resource temporarily unavailable; try again later

...

30


PiCameraMMALError: no buffers available: Resource temporarily unavailable; try again later

31


PiCameraMMALError: no buffers available: Resource temporarily unavailable; try again later

Camera Stopped
